In [1]:
suppressPackageStartupMessages({
    library(tidymodels)
    library(treesnip)
    library(tidyverse)
    library(keras)
    library(tensorflow)
    #library(lime)
})

In [2]:
seed <- 1923

In [3]:
set.seed(1)

label <- 'income'

iris1.5 <- ISLR::Default %>% mutate(default = case_when(default == 'No' ~ 0,
                                             default == 'Yes' ~ 1)) %>% select(-student)

iris1.5 <- iris1.5 %>% mutate(default = factor(default,levels = 0:1))

willsee <- caret::createDataPartition(iris1.5[[label]],p = .7,list = TRUE) %>% unlist

iris2 <- iris1.5[willsee,]

iris3 <- iris1.5[-willsee,]

folds_ <- caret::createFolds(iris2[[label]],k = 5)

folds <- 1:length(folds_)

errors <- c()

predictions <- list()

iris2 <- iris2 %>% mutate(default = as.integer(as.character(default)))
iris3 <- iris3 %>% mutate(default = as.integer(as.character(default)))

i <- 1

train_indices_dum <- setdiff(folds,i)
test_indices_dum <- i

train_set_dum <- iris2[unlist(folds_[train_indices_dum]),]
test_set_dum <- iris2[unlist(folds_[test_indices_dum]),]


In [4]:
input_neurons <- ncol(train_set_dum) - 1
output_neurons <- 2

nofsample <- nrow(train_set_dum)

alpha <- 4

In [5]:
numofneuron <- nofsample / (alpha * (input_neurons + output_neurons))

numofneuron <- numofneuron - output_neurons

In [49]:
layer_dist1 <- base::seq(from = numofneuron,to = output_neurons,length.out = 5)
layer_dist <- layer_dist1[-1]/base::sum(layer_dist1[-1])*numofneuron
layer_disto <- layer_dist %>% ceiling %>% .[1:4]

#randy <- sample(1:1000,size = 8)

#(numofneuron*(randy/sum(randy)) )%>% ceiling

In [50]:
layer_disto

[1] 173 116  59   2

In [16]:
f1 <- function(y_true, y_pred){
    y_pred = k_round(y_pred)
    
    tp = k_sum(k_cast(y_true*y_pred, 'float'), axis=1)
    tn = k_sum(k_cast((1-y_true)*(1-y_pred), 'float'), axis=1)
    fp = k_sum(k_cast((1-y_true)*y_pred, 'float'), axis=1)
    fn = k_sum(k_cast(y_true*(1-y_pred), 'float'), axis=1)

    p = tp / (tp + fp + k_epsilon())
    r = tp / (tp + fn + k_epsilon())

    f1 = 2*p*r / (p+r+k_epsilon())
    f1 = tf$where(tf$math$is_nan(f1), tf$zeros_like(f1), f1)
    return(k_mean(f1,axis = 1))
}

f1_loss <- function(y_true, y_pred){
    #y_pred = k_round(y_pred)
    tp = k_sum(k_cast(y_true*y_pred, 'float'), axis=1)
    tn = k_sum(k_cast((1-y_true)*(1-y_pred), 'float'), axis=1)
    fp = k_sum(k_cast((1-y_true)*y_pred, 'float'), axis=1)
    fn = k_sum(k_cast(y_true*(1-y_pred), 'float'), axis=1)

    p = tp / (tp + fp + k_epsilon())
    r = tp / (tp + fn + k_epsilon())

    f1 = 2*p*r / (p+r+k_epsilon())
    f1 = tf$where(tf$math$is_nan(f1), tf$zeros_like(f1), f1)
    return(1 - k_mean(f1,axis = 1))
}

In [17]:
#model %>% compile(
#  loss = f1_loss,
#  optimizer = optimizer_adam(learning_rate = 0.01),
#  metrics = c(f1)
#)

In [21]:
train_indices <- setdiff(folds,i)
    test_indices <- i
    
    train_set <- iris2[unlist(folds_[train_indices]),]
    test_set <- iris2[unlist(folds_[test_indices]),]
    

In [35]:
label

[1] "income"

In [51]:
pls <- list()

for(i in folds){
    train_indices <- setdiff(folds,i)
    test_indices <- i
    
    train_set <- iris2[unlist(folds_[train_indices]),]
    test_set <- iris2[unlist(folds_[test_indices]),]
    
    tarif <- recipe(income ~ .,data = train_set) %>% 
    step_normalize(all_numeric_predictors()) %>%
    #step_encode_mean(all_nominal_predictors(),outcome = 'default') %>%
    step_dummy(all_nominal_predictors()) %>%
    prep
    
    target_data <- bake(tarif,iris3)
    
    train_set <- bake(tarif,train_set)
    test_set <- bake(tarif,test_set)
    
    train_y <- train_set[,label] %>% as.matrix# %>% to_categorical
    test_y <- test_set[,label]%>% as.matrix #%>% to_categorical
    test_y2 <- test_set[,label]%>% as.matrix
    
    train_x <- train_set[, -which(names(train_set) == label)]%>% as.matrix
    test_x <- test_set[, -which(names(test_set) == label)]%>% as.matrix
    
    target_data <- target_data[, -which(names(train_set) == label)]%>% as.matrix
    
    tf$random$set_seed(seed = seed)

    reticulate::py_set_seed(seed = seed)

    model <- keras_model_sequential()
    
    randy <- sample(1:1000,size = 8)

    unites <- (numofneuron*(randy/sum(randy)) )%>% ceiling

    model %>%
    layer_dense(units = layer_disto[1], activation = 'relu', input_shape = c(2)) %>%
    layer_dense(units = layer_disto[2], activation = 'relu') %>%
    layer_dense(units = layer_disto[3], activation = 'relu') %>%
    layer_dense(units = layer_disto[4], activation = 'relu') %>%
    #layer_dense(units = layer_disto[5], activation = 'relu') %>%
    #layer_dense(units = layer_disto[6], activation = 'relu') %>%
    #layer_dense(units = layer_disto[7], activation = 'relu') %>%
    #layer_dense(units = layer_disto[8], activation = 'relu') %>%
    layer_dense(units = 1,activation = 'linear')
    
    model %>% compile(
      loss = 'mean_squared_error',
      optimizer =optimizer_rmsprop(learning_rate = 0.0001),
      metrics = 'mse'
    )
    
    wgh <- sqrt(sum(test_y)/sum(test_y == 0))
    
    history <- model %>% fit(
      train_x, train_y, 
      epochs = 150, batch_size = 128, 
      validation_data = list(test_x,test_y),
      callbacks = list(callback_early_stopping(
                        monitor = "val_loss",
                        patience = 10,
                        verbose = 1,
                        mode = "auto",
                        restore_best_weights = TRUE
                        )
    )
    )

    preds <- model %>% predict(test_x) # %>% 
                        #as.data.frame %>%
                        #mutate('class'=names(.)[apply(., 1, which.max)]) %>%
                        #mutate(class = case_when(class == 'V1' ~ 0,
                        #                         class == 'V2' ~ 1)) %>%
                        #select(class) %>%
                        #pull
    
    error <- yardstick::rmse_vec(truth = as.numeric(test_y2),estimate = as.numeric(preds))
    
    preds_target <- model %>% predict(target_data)  #%>% 
                        #as.data.frame %>%
                        #mutate('class'=names(.)[apply(., 1, which.max)]) %>%
                        #mutate(class = case_when(class == 'V1' ~ 0,
                        #                         class == 'V2' ~ 1)) %>%
                        #select(class) %>%
                        #pull
    
    IRdisplay::display('**************----------**************')
    IRdisplay::display(sprintf('Fold %s RMSE : %s',i,round(error,2)))
    errors[i] <- error
    
    pls[[i]] <- history %>% plot
    
    predictions[[i]] <- preds_target
}
IRdisplay::display('**************----------**************')
IRdisplay::display(sprintf('CV Mean RMSE : %s',mean(errors,na.rm = T)))

dson <- bake(tarif,iris2)

strain_y <- dson[,label] %>% as.matrix %>% to_categorical

strain_x <- dson[, -which(names(dson) == label)]%>% as.matrix
    

#historyson <-  model %>% fit(
#      strain_x, strain_y, 
#      epochs = 100, batch_size = 128, 
#      validation_data = list(test_x,test_y),
#      callbacks = list(callback_early_stopping(
#                        monitor = "val_loss",
#                        patience = 20,
#                        verbose = 1,
#                        mode = "auto",
#                        restore_best_weights = TRUE
#                        )
#    )
#    )

[1] "**************----------**************"

[1] "Fold 1 RMSE : 13473.14"

[1] "**************----------**************"

[1] "Fold 2 RMSE : 13394.87"

[1] "**************----------**************"

...

[1] "Fold 4 RMSE : 13324.88"

[1] "**************----------**************"

[1] "Fold 5 RMSE : 13226.51"

[1] "**************----------**************"

[1] "CV Mean RMSE : 13329.7945289611"

In [53]:
mod <- lm(income ~ default + balance,iris2)

In [54]:
lmpred <- predict(mod,iris3)

iris3 %>%
mutate(pred = lmpred) %>%
rmse(truth = income,estimate = pred)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
rmse,standard,13160.37


In [55]:
#pls %>% lapply(function(x) x + xlim(0,150))

In [56]:
#dsson <- bake(tarif,iris3)

#sstrain_y <- dsson[,label] %>% as.matrix %>% to_categorical

#sstrain_x <- dsson[, -which(names(dsson) == label)]%>% as.matrix

#sstrain_y2 <- dsson[,label] %>% as.matrix 

In [57]:
#prds <- predict(model,sstrain_x) %>% apply(1,which.max) - 1

In [58]:
#sstrain_y2 %>%
#as.data.frame %>% 
#bind_cols(a = prds) %>%
#mutate_all(as.factor) %>%
#f_meas(truth = default,estimate = a,event_level = "second")

In [59]:
all <- as.data.frame(predictions) %>% mutate_if(is.factor,function(x) as.numeric(as.character(x)))

In [60]:
kk <- rowMeans(all) 
#kk <- ifelse(kk >= 0.5,1,0)

In [61]:
kl <- iris3[,label]

In [62]:
keras_preds <- kk

In [63]:
yardstick::rmse_vec(truth = kl,estimate = kk)

[1] 13251.9